# Implementation of simple simplex algorithm:

In [4]:
#A function that returns a simplex array. It takes an object of matrix type and returns the same.

def simplex_table(m):
    base_values = []
    for i in range(m.ncols()-1):
        if sum(map(abs, vector(m[2:,0:m.ncols()-1][:,i]))) == 1 and max(vector(m[2:,0:m.ncols()-1][:,i])) == 1:
            base_values.append(m[1,i])
    
    reduced_cost = []
    for i in range(m.ncols()):
        val = vector(base_values).dot_product(vector(m[2:,i])) + (-1)*m[1,i]
        reduced_cost.append(val)
        
    m = m.rows()
    m.append(reduced_cost)
    return(matrix(m))

#The function that calculates the central element of the matrix, returns its positions in the array.

def get_point(m):
    row = list(m.row(m.nrows()-1))
    max_value = max(row)
    index_col = row.index(max_value)
    val1 = m[2:,index_col].transpose()
    val2 = m[2:,m.ncols()-1].transpose()
    values = []
    
    for i in range(val1.ncols()-1):
        if val1[0,i] > 0:
            values.append(val2[0,i]/val1[0,i])
        else:
            values.append(-1)
    
    min_value = min([i for i in values if i > 0])
    index_row = values.index(min_value)
    get_index = [index_row + 2, index_col]
    
    return get_index

#A function that transforms a matrix with respect to a given point using the Gaussian method. It takes matrix-like object
#and point indices. 

def Gauss(A,i,j):
    A[i]=A[i]/A[i,j]
    for k in range(2,A.nrows()):
        if k!=i:
            A[k]=A[k]-A[k,j]*A[i]
    return A

#Function that reads the value of the function and the vertices of the points from the matrix with negative reduced costs.
#It takes matrix object and returns list of strings.

def results(m, opt):
    txt = []
    values_column = vector(m[2:m.nrows()-1,m.ncols()-1].transpose())

    for i in range(m.ncols()-1):
        if m[m.nrows()-1, i] == 0:
            txt.append(f'{m[0,i]} = {vector(m[2:m.nrows()-1,i].transpose()).dot_product(values_column)}')
        else: 
            txt.append(f'{m[0,i]} = 0')
    if opt == "min":
        txt.append(f'Min value of function is equal to:  {m[m.nrows()-1, m.ncols()-1]}')
    if opt == "max":
        txt.append(f'Max value of function is equal to:  {m[m.nrows()-1, m.ncols()-1] * (-1)}')
    return txt

#Function that implements the algorithm. It takes matrix object and returns list of strings. 

def simplex(m, opt):
    
    simplex_matrix = simplex_table(m) #create a simplex table 
    
    while max(list(vector(simplex_matrix[simplex_matrix.nrows()-1,:]))) != 0: #loop works until the last row in matrix contains non positive values
        point = get_point(simplex_matrix) #finding a center point
        simplex_matrix = Gauss(simplex_matrix, point[0], point[1]) #Gaussian transformation with respect to the found point
        
    return results(simplex_matrix, opt) #returns answer 

Maximize function $f=x+y+z$ with constraints:

$$\left\{\begin{array}{l}
   x-y-2z\leq 5\\
   x+2y-3z\leq 3\\
   x+2y+6z\leq 5\\
   2x+y-5z\leq 8\\
   x,y,z\geq 0\ .
  \end{array}\right.$$

Define the matrix, we turn inequalities into equalities by adding some a,b,c,d to equations:

In [2]:
var('a','b','c','d','x','y','z','f')

M = matrix(
[
[a,b,c,d,x,y,z,f],
[0,0,0,0,-1,-1,-1,0],
[1,0,0,0,1,-1,-2,5],
[0,1,0,0,1,2,-3,3],
[0,0,1,0,1,2,6,5],
[0,0,0,1,2,1,-5,8],
]
)

show(M)

[ a  b  c  d  x  y  z  f]
[ 0  0  0  0 -1 -1 -1  0]
[ 1  0  0  0  1 -1 -2  5]
[ 0  1  0  0  1  2 -3  3]
[ 0  0  1  0  1  2  6  5]
[ 0  0  0  1  2  1 -5  8]

Function call:

In [6]:
simplex(M, 'max')

['a = 16/9',
 'b = 0',
 'c = 0',
 'd = 16/9',
 'x = 11/3',
 'y = 0',
 'z = 2/9',
 'Max value of function is equal to:  35/9']